In [0]:
import numpy as np
import pandas as pd
import os
import pickle
os.chdir("C:/Users/yieun_2/Desktop/project/finance")
with open('cherry.pickle', 'rb') as f:
    df = pickle.load(f)  #df: 질문 제목, 질문 내용, 카테고리, 답변, url, summary(docsim, tfidf_co의 input), final(jaccard의 input)

In [0]:
# 임베딩 모델 로드
from sklearn.externals import joblib
embedding_model = joblib.load("skipgram_ver1_1.sav")

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [0]:
# 예측 모델 로드
filename = 'mlp_clf1.sav'
classify_model = pickle.load(open(filename, 'rb'))

C:\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
# module import

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
okt = Okt()

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        pos = ['{}/{}'.format(word,tag) for word, tag in pos]
        return pos
    
my_tokenizer = MyTokenizer(Okt())
tfidf = TfidfVectorizer(tokenizer = my_tokenizer)

from sklearn.metrics.pairwise import linear_kernel
from scipy import spatial
import re

In [0]:
def jaccard(sen1, sen2): #자카드 유사도 
    a = set(sen1) 
    b = set(sen2) 
    aUb = a.union(b) 
    aNb = a.intersection(b) 
    p = len(aNb) / len(aUb) 
    return p

    
def get_recom(a,b):    #tf_idf행렬을 이용한 cosine 유사도
    data = pd.DataFrame(data=[a,b], index=range(0,2),columns = ["o"])
    tfidf_matrix = tfidf.fit_transform(data["o"])
    
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    final = cosine_sim[0][1]
    return final

def avg_feature_vector(sentence, model, num_features, index2word_set):  #w2v 문장 임베딩 벡터
    words = str(sentence).split(" ")
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
    return feature_vec
    
def docsim(a,b):  #w2v문장 임베딩 벡터를 이용한 문장 간 유사도
    index2word_set = set(embedding_model.wv.index2word)
    s1_afv = avg_feature_vector(a, model=embedding_model, num_features=100, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(b, model=embedding_model, num_features=100, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    return sim

def calculate(df, input_string):  # 각 데이터에 대해 계산 #가중치 앙상블
    tmp = okt.nouns(input_string)
    df['jaccard'] = df["final"].apply(lambda x : jaccard(tmp, x))
    df['doc_sim'] = df["summary"].apply(lambda x: docsim(input_string, x))
    df['tfidf_co'] = df["summary"].apply(lambda x: get_recom(x,input_string))
    df["similarity"] = 0.4*df["doc_sim"]+0.4*df["jaccard"]+0.2*df["tfidf_co"]
    df = df.sort_values(by = 'similarity', ascending = False)
    return df[:6][['title', 'question', 'category', 'answer', 'url', 'jaccard','doc_sim','tfidf_co','similarity']]


def preprocessing(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9 .a-zA-Z ]+', '', str(text))
    return text


def receive_input():  #input 받은 후 전처리
    title = input('제목을 입력해주세요.')
    content = input('내용을 입력해주세요.')
    
    title=preprocessing(title)
    content=preprocessing(content)

    res = title + content
    return res


def main():
    res = receive_input()
    index2word_set = set(embedding_model.wv.index2word)
    res_vec = avg_feature_vector(res, model=embedding_model, num_features=100, index2word_set=index2word_set)
    cl_pred = classify_model.predict(res_vec.reshape(1,-1))  #인풋 데이터 레이블 예측
    clustered_df = df.loc[df.cluster == cl_pred[0]]  
    return calculate(clustered_df, res)

In [0]:
import time
import warnings
warnings.filterwarnings(action='ignore')
start_time = time.time()
main()

제목을 입력해주세요.보안 카드 분실
내용을 입력해주세요.보안 카드를 분실했는데, 재발급 받을 수 있나요?


,title,question,category,answer,url,jaccard,doc_sim,tfidf_co,similarity
11472,농협 보안카드 분실,안녕하세요 고 남학생인데 봄쯤에 학교에 농협 직원분들이 오셔서 학생증 만드는데 필요...,"뱅킹, 금융업무",까 네이버지식파트너 NH농협입니다 문의하신 보안카드 재발급 안내 해 드리겠습니다 ...,https://kin.naver.com/qna/mydetail.nhn?dirId=4...,0.137931,0.754593,0.448352,0.446680
6391,보안카드 분실이요,농협 보안카드를 분실했는데 필요한 준비물이 있나요그 분실한 카드 통장을 들고 가야 하나요,"뱅킹, 금융업무",보안카드를 분실하셨다면 본인이 직접 은행에 신분증 지참후 내점하셔야 합니다 ...,https://kin.naver.com/qna/mydetail.nhn?dirId=4...,0.176471,0.629505,0.557709,0.433932
995,기업은행 OTP 카드 만료인데 타행 OTP 발급 받을 수 있나요,현재 기업은행 OTP 카드 사용중이고 얼마전에 기간 만료 안내가 왔는데 시간도 ...,"뱅킹, 금융업무",기존OTP가 있다해도 다른 OTP등록후 사용가능합니다 당사의 경우 OTP등록...,https://kin.naver.com/qna/mydetail.nhn?dirId=4...,0.071429,0.841785,0.151796,0.395644
12250,농협 보안카드 재발급,제가 농협 보안카드를 재발급 받으려고 하는데 재발급 받으려면 준비물이 어떤게 필요하...,"뱅킹, 금융업무",까 네이버지식파트너 NH농협입니다 문의주신 농협 보안카드 재발급 관련 안내드리겠...,https://kin.naver.com/qna/mydetail.nhn?dirId=4...,0.157895,0.626865,0.319358,0.377775
11865,농협 보안카드 잃어버림,재발급 받으려면 은행 가야하나요 뭐 필요하나요 저성인임,"뱅킹, 금융업무",까네이버지식파트너 NH농협입니다 신분증 지참후 가까운 농협으로 방문하시면 보안카...,https://kin.naver.com/qna/mydetail.nhn?dirId=4...,0.235294,0.588494,0.233582,0.376232
11627,농협 통장 재발급 받으려고합니다,제가 농협 체크카드랑 통장이 있습니다 그런데 통장을 분실해버려서 통장을 재발급 받...,"뱅킹, 금융업무",까 네이버지식파트너 NH 농협 입니다 문의주신 통장 재발급 관련 안내 해 드리겠습...,https://kin.naver.com/qna/mydetail.nhn?dirId=4...,0.125000,0.725753,0.156383,0.371578


In [0]:
print("--- %s seconds ---" %(time.time() - start_time))   

--- 15.674705982208252 seconds ---


In [0]:
df.iloc[11472,0]

'까 네이버지식파트너 NH농협입니다  문의하신 보안카드 재발급 안내 해 드리겠습니다   만 세 이상 미성년자인 경우 본인 또는 법정대리인이 서류 지참후 가까운 영업점으로 방문하여 보안카드 재발급 가능합니다    본인 방문 시  신분증   법정대리인 친권자  방문 시  방문하는 법정대리인 신분증  미성년자 기준의 특정 또는 상세 기본증명서  미성년자와 관계가 표기된 가족관계증명서 일반증명서에 표기 안된 경우 상세증명서 지참   다른 궁금한 사항이 있으시거나  유선 상담을 원하시는 경우 고객행복센터       으로 평일 시시까지 문의 바랍니다   이용해 주셔서 감사합니다  항상 건강하시고 행복한 하루 되십시오 '